In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
import tqdm
import glob
import os
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
with open('DataSplit.pickle', 'rb') as handle:
    DataSplit = pickle.load(handle)

In [3]:
class Senz3DDataset(Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels
        self.GT_labels = {'1': 5, '2': 2, '3': 3, '4': 5, '5': 0, '6': 2, 
                          '7': 1, '8': 4, '9': 3, '10': 1, '11': 1}
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        img = cv2.imread("..\\." + self.paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4)).astype(np.uint8)
        
        label = torch.tensor(self.GT_labels[(self.labels[idx])])
        
        return {'img': img, 'label': label}

In [4]:
TestDataset = Senz3DDataset(list(DataSplit['test'].keys()), list(DataSplit['test'].values()))
TestDataLoader = DataLoader(TestDataset, batch_size=1, shuffle=False)
person = [path.split('\\')[3] for path in (TestDataset.paths)]
gesture = [path.split('\\')[4] for path in (TestDataset.paths)]

In [ ]:
def eval_test(test_loader, model, loss_fn, device):
    test_total = 0
    test_correct = 0
    preds = [] 
    GT = []
    model.eval()
    with torch.no_grad():
        for i, sample in enumerate(test_loader):

            # move inputs to device
            if torch.cuda.is_available():
                x = sample['img'].permute(0, 3, 1, 2).float().to(device)
                y = sample['label'].to(device)

            output = model.forward(x)
            
            # Calculate accuracy/loss
            _, y_hat = torch.max(output, dim=1)
            preds.append(y_hat.cpu().data.numpy()[0])
            GT.append(y_hat.cpu().data.numpy()[0])
            
            batch_correct = torch.sum(y_hat == y)
            test_correct += batch_correct
            test_total += x.shape[0]
    test_acc = test_correct.float()/test_total
    print(f'Test Acc: {test_acc}')
    return test_acc, preds, GT

In [ ]:
import torchvision.models as models
model_name = 'resnext50'
model = models.resnext50_32x4d()
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024, bias=True),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 64),
    nn.ReLU(),
    nn.Linear(64,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnext50_preds, resnext50_GT = eval_test(TestDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

In [ ]:
model_name = 'resnet152'
model = models.resnet152()
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024, bias=True),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 64),
    nn.ReLU(),
    nn.Linear(64,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnet152_preds, resnet152_GT = eval_test(TestDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

In [ ]:
model_name = 'resnet18'
model = models.resnet18()
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=1000, bias=True),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500,50),
    nn.ReLU(),
    nn.Linear(50,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnet18_preds, resnet18_GT = eval_test(TestDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

In [ ]:
Test_Model_Outputs = {'Person (S#)': person, 'Gesture (G#)': gesture, "GT Label": resnet18_GT,
                      'Resnet18': resnet18_preds , 'Resnet152': resnet152_preds,
                      'Resnext50': resnext50_preds}

## Do Validation

In [ ]:
ValDataset = Senz3DDataset(list(DataSplit['val'].keys()), list(DataSplit['test'].values()))
ValDataLoader = DataLoader(ValDataset, batch_size=1, shuffle=False)
person = [path.split('\\')[3] for path in (ValDataset.paths)]
gesture = [path.split('\\')[4] for path in (ValDataset.paths)]

In [ ]:
model_name = 'resnext50'
model = models.resnext50_32x4d()
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024, bias=True),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 64),
    nn.ReLU(),
    nn.Linear(64,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnext50_preds, resnext50_GT = eval_test(ValDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

import torchvision.models as models
model_name = 'resnet152'
model = models.resnet152()
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024, bias=True),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 64),
    nn.ReLU(),
    nn.Linear(64,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnet152_preds, resnet152_GT = eval_test(ValDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

model_name = 'resnet18'
model = models.resnet18()
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=1000, bias=True),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500,50),
    nn.ReLU(),
    nn.Linear(50,6),
    nn.Softmax(dim=1),    
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("saved_models/" + model_name + ".pth"))
model.to(device)
model.eval()

a = time.time()
epoch_test_accuracy, resnet18_preds, resnet18_GT = eval_test(ValDataLoader, model, nn.CrossEntropyLoss(), device)
print("Time elapsed for evaluating test set is: ", time.time() - a)

In [ ]:
Val_Model_Outputs = {'Person (S#)': person, 'Gesture (G#)': gesture, "GT Label": resnet18_GT,
                     'Resnet18': resnet18_preds , 'Resnet152': resnet152_preds,
                     'Resnext50': resnext50_preds}

## Build Dictionary/Dataframe

In [ ]:
with open('Val_Model_Outputs.pickle', 'wb') as handle:
    pickle.dump(Val_Model_Outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('Test_Model_Outputs.pickle', 'wb') as handle:
    pickle.dump(Test_Model_Outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)